# Bibliotecas

In [ ]:
import numpy as np

# Abrindo dados

In [ ]:
x = np.float64(np.load(open('data/cnn/x.npy', 'rb')))
y = np.int8(np.load(open('data/cnn/y.npy', 'rb')))

In [ ]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

# Criar Modelo

In [ ]:
from keras.applications import EfficientNetV2M
from keras import layers, models

input = layers.Input(shape=(501, 6, 1))
reshape_layer = layers.Cropping2D(cropping=((15, 0), (0, 0)))(input)
reshape_layer = layers.Reshape((54, 54, 1))(reshape_layer)
reshape_layer = layers.Concatenate()([reshape_layer, reshape_layer, reshape_layer])

importedNet = EfficientNetV2M(include_top=True, input_shape=(54, 54, 3), weights=None, classes=3)

modelo = importedNet(reshape_layer)
# modelo = layers.BatchNormalization()(modelo)
# modelo = layers.Flatten()(modelo)
# modelo = layers.Dropout(0.5)(modelo)
# modelo = layers.Dense(100, 'relu')(modelo)
# modelo = layers.Dropout(0.5)(modelo)
# modelo = layers.Dense(100, 'relu')(modelo)
# modelo = layers.Dense(3, 'softmax')(modelo)

modelo = models.Model(input, modelo)
modelo.compile(loss='categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

# Treinar Modelo

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/transferLearning/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=20, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('/mnt/c/Users/aurel/OneDrive/Documentos/Projetos/tradingPredictor/models/TransferLearningmodel7.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=500,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)